In [8]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2


In [9]:
# Load the saved model
model = load_model('vgg16_custom_model.h5')
print("Model loaded successfully.")

Model loaded successfully.


In [10]:
# Initialize the face detector using OpenCV's Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [11]:
# Function to detect faces, predict mask status, and label each face in the image
def predict_and_label_faces(model, img_path):
    # Load the image
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale for face detection

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Check if any faces are detected
    if len(faces) == 0:
        print("No faces detected in the image.")
        return

    # Define class labels
    class_labels = ['With Mask', 'Without Mask']

    # Process each detected face
    for idx, (x, y, w, h) in enumerate(faces):
        # Crop and preprocess the face
        face_img = img[y:y + h, x:x + w]
        face_resized = cv2.resize(face_img, (224, 224))
        face_array = image.img_to_array(face_resized) / 255.0
        face_array = np.expand_dims(face_array, axis=0)

        # Predict mask status
        prediction = model.predict(face_array)
        predicted_class = np.argmax(prediction, axis=1)[0]
        
        # Determine label and color based on prediction
        prediction_label = class_labels[predicted_class]
        color = (0, 255, 0) if prediction_label == "With Mask" else (0, 0, 255)  # Green for "With Mask," Red for "Without Mask"
        
        # Draw a bounding box around the face
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        
        # Display the mask status label on top of the bounding box
        label_text = f"{prediction_label}"
        cv2.putText(img, label_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # Show the image with labeled faces
    cv2.imshow("Mask Detection", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [13]:
# Example usage of the function
img_path = "C:/ALL IN ONE/face mask/somnath with mask.jpg"  # Replace with your image path
predict_and_label_faces(model, img_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 525ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2

# Load the saved model
model = load_model('vgg16_custom_model.h5')
print("Model loaded successfully.")

# Initialize the face detector using OpenCV's Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to predict and label faces in an image path
def predict_from_image_path(model, img_path):
    # Load the image
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        print("No faces detected.")
        return

    class_labels = ['With Mask', 'Without Mask']

    for (x, y, w, h) in faces:
        face_img = img[y:y + h, x:x + w]
        face_resized = cv2.resize(face_img, (224, 224))
        face_array = image.img_to_array(face_resized) / 255.0
        face_array = np.expand_dims(face_array, axis=0)

        prediction = model.predict(face_array)
        predicted_class = np.argmax(prediction, axis=1)[0]
        label = class_labels[predicted_class]
        color = (0, 255, 0) if label == "With Mask" else (0, 0, 255)

        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    cv2.imshow("Image Mask Detection", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Function to predict mask status in real-time from the webcam
def predict_from_camera(model):
    cap = cv2.VideoCapture(0)
    class_labels = ['With Mask', 'Without Mask']

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            face_img = frame[y:y + h, x:x + w]
            face_resized = cv2.resize(face_img, (224, 224))
            face_array = image.img_to_array(face_resized) / 255.0
            face_array = np.expand_dims(face_array, axis=0)

            prediction = model.predict(face_array)
            predicted_class = np.argmax(prediction, axis=1)[0]
            label = class_labels[predicted_class]
            color = (0, 255, 0) if label == "With Mask" else (0, 0, 255)

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        cv2.imshow("Real-Time Mask Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function to choose between real-time detection or image path detection
def main():
    choice = input("Choose detection mode:\n1. Real-Time Camera\n2. Image Path\nEnter 1 or 2: ")

    if choice == '1':
        print("Starting real-time camera detection...")
        predict_from_camera(model)
    elif choice == '2':
        img_path = input("Enter the image path: ")
        print(f"Detecting faces in {img_path}...")
        predict_from_image_path(model, img_path)
    else:
        print("Invalid choice. Please enter 1 or 2.")

# Run the main function
main()


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'vgg16_trained_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)